### Metadata Replacement + Node Sentence Window
https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/MetadataReplacementDemo/

In [1]:
# %pip install llama-index-embeddings-openai
# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-llms-openai
# %pip install llama-index
# # %pip install --upgrade llama-index-core llama-index-llms-openai llama-index-embeddings-openai llama-index-embeddings-huggingface
# %pip install llamaapi

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import openai

OpenAI(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x442499090>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x15c0de050>, completion_to_prompt=<function default_completion_to_prompt at 0x15c17e200>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='gpt-3.5-turbo', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='sk-zk2e65c7b3aa4d285237b360e338e2c174066b0a70704f61', api_base='https://api.openai.com/v1', api_version='')

In [3]:
from llama_index.llms.openai import OpenAI as li_OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter

/Users/hudanyunsheng/miniforge3/envs/jpfaq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [5]:
# base node parser is a sentence splitter
text_splitter = SentenceSplitter()

In [6]:
os.environ["OPENAI_API_KEY"] = "sk-zk2e65c7b3aa4d285237b360e338e2c174066b0a70704f61"
os.environ["llama_API_KEY"] = "LL-8UzIPeKYxLIbdX9x66MQ7tKWb16lOA5FWWkViLhD54piGPO3WpLkRlikijI6DNEU"

In [19]:
llm_openAI = li_OpenAI(model="gpt-3.5-turbo", temperature=0.1)
print(type(llm_openAI))

from llamaapi import LlamaAPI
llm = LlamaAPI(os.environ.get("llama_API_KEY"))
print(type(llm))

llm_client = li_OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"), api_base="https://api.zhizengzeng.com/v1", model="gpt-3.5-turbo", temperature=0.1
)
print(type(llm_client))

<class 'llama_index.llms.openai.base.OpenAI'>
<class 'llamaapi.llamaapi.LlamaAPI'>
<class 'llama_index.llms.openai.base.OpenAI'>


In [8]:
# openAIclient = openai.OpenAI(
#     api_key = os.environ.get("OPENAI_API_KEY"),
#     base_url = "https://api.zhizengzeng.com/v1"
# )
# type(openAIclient)

openai.OpenAI

In [9]:
# (this step needs vpn)
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2", max_length=512
)

In [20]:
# llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

# llm = LlamaAPI(os.environ.get("llama_API_KEY"))
# llm = OpenAI(
#     api_key = os.environ.get("llama_API_KEY"),
#     base_url = "https://api.llama-api.com"
# )


llm = llm_client
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [46]:
# !curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

In [10]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./IPCC_AR6_WGII_Chapter03.pdf"]
).load_data()

In [11]:
nodes = node_parser.get_nodes_from_documents(documents)
base_nodes = text_splitter.get_nodes_from_documents(documents)

In [14]:
from llama_index.core import VectorStoreIndex

sentence_index = VectorStoreIndex(nodes)

In [15]:
base_index = VectorStoreIndex(base_nodes)

In [21]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
window_response = query_engine.query(
    "What are the concerns surrounding the AMOC?"
)
print(window_response)

There is low confidence in quantifying AMOC changes in the 20th century due to disagreement in reconstructed and simulated trends. Additionally, the relative contributions of internal variability, natural forcing, and anthropogenic forcing to AMOC change cannot be determined due to the short duration of direct observational records since the mid-2000s. Despite projections indicating a decline in AMOC over the 21st century for all scenarios, there is high confidence that an abrupt collapse is not expected before 2100.


'''
Unknown model 'llama3.1'. Please provide a valid OpenAI model name in: gpt-4, gpt-4-32k, gpt-4-1106-preview, gpt-4-0125-preview, gpt-4-turbo-preview, gpt-4-vision-preview, gpt-4-1106-vision-preview, gpt-4-turbo-2024-04-09, gpt-4-turbo, gpt-4o, gpt-4o-2024-05-13, gpt-4o-mini, gpt-4o-mini-2024-07-18, gpt-4-0613, gpt-4-32k-0613, gpt-4-0314, gpt-4-32k-0314, gpt-3.5-turbo, gpt-3.5-turbo-16k, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, gpt-3.5-turbo-0613, gpt-3.5-turbo-16k-0613, gpt-3.5-turbo-0301, text-davinci-003, text-davinci-002, gpt-3.5-turbo-instruct, text-ada-001, text-babbage-001, text-curie-001, ada, babbage, curie, davinci, gpt-35-turbo-16k, gpt-35-turbo, gpt-35-turbo-0125, gpt-35-turbo-1106, gpt-35-turbo-0613, gpt-35-turbo-16k-0613
'''

In [116]:
from openai import OpenAI
client = OpenAI(api_key="sk-zk2e65c7b3aa4d285237b360e338e2c174066b0a70704f61", 
                base_url="https://api.zhizengzeng.com/v1")

resp = client.chat.completions.create(
    model="SparkDesk",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "你是哪个公司开发的什么模型？"}
    ]
)
resp

ChatCompletion(id='', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=None))], created=1722901905, model=None, object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=0, prompt_tokens=0, total_tokens=0), code=0, msg='ok')

In [133]:
client = OpenAI(api_key="sk-zk2e65c7b3aa4d285237b360e338e2c174066b0a70704f61", base_url="https://api.zhizengzeng.com/v1/")
resp = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "圆周率的前10位"}
    ]
)

In [134]:
print(reszp)

ChatCompletion(id='chatcmpl-9t20YzNoM6KNjZxO5O3bAGRV7W5Km', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='圆周率的前10位小数为3.1415926535。', role='assistant', function_call=None, tool_calls=None))], created=1722902786, model='gpt-3.5-turbo', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=25, total_tokens=43), code=0, msg='ok')
